# Objective

<p>
    <span style='font-family:Arial'>
    Classify spam text message
    </span>
</p> 

In [ ]:
import numpy as np 
import pandas as pd 
import nltk as nlp
import re
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [ ]:
# load data
data = pd.read_csv("../input/spam-text-message-classification/SPAM text message 20170820 - Data.csv")

In [ ]:
data.head()


In [ ]:
# convert spam to 1, not spam to 0
data["Category"] = [1 if category == "spam" else 0 for category in data["Category"]]

In [ ]:
data.head()

In [ ]:
description_list = []
for description in data["Message"]:
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()   
    description = nlp.word_tokenize(description)
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description) 

In [ ]:
max_features = 3000 
count_vectorizer = CountVectorizer(max_features = max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print("The most used {} words: {}".format(max_features,count_vectorizer.get_feature_names()))

In [ ]:
y = data.iloc[:,0].values   # male or female classes
x = sparce_matrix

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.1, random_state = 42)

In [ ]:
nb = GaussianNB()
nb.fit(x_train,y_train)
print("Gaussian NB model Accuracy: {}".format(nb.score(x_test,y_test)))

In [ ]:
lr = LogisticRegression(max_iter = 200)
lr.fit(x_train,y_train)
print("Logistic Regression model Accuracy: {}".format(lr.score(x_test,y_test)))

In [ ]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
print('KNN (K=3) model accuracy is: ',knn.score(x_test,y_test))

In [ ]:
x_train = x_train.reshape(5014,3000,1)
x_test = x_test.reshape(558,3000,1)

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics=["accuracy"])

regressor.fit(x_test, y_test, epochs = 3, batch_size = 32)
